**NEGATIVE WORDS GENERATOR**

In [ ]:
pip install transformers datasets torch # Install prerequisites

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
import torch

file_path = "/negative.txt"  # File path to the data file

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
### 1. Load & Process Your Dataset ###
def load_negative_words(file_path):
    words = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            parts = line.strip().split()  # Split line by spaces
            if len(parts) == 2:  # Ensure correct format (word, score)
                word, score = parts
                try:
                    score = float(score)  # Convert score to float
                    if score < 0:  # Keep only negative words
                        words.append(word)
                except ValueError:
                    continue  # Skip invalid lines
    return words
negative_words = load_negative_words(file_path) # Load dataset from file

# Convert list to dataset format
dataset = Dataset.from_dict({"text": negative_words})

In [ ]:
### 2️. Load GPT-2 Tokenizer & Model ###
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", is_split_into_words=False) # I tried using AutoTokenizer here instead. Same results as with GTP2 Tokenizer in pos_generator.
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
model = GPT2LMHeadModel.from_pretrained("distilgpt2")

In [ ]:
### 3️. Tokenize the Data ###
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=10) # Tokenize the data

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Data collator for training (helps with batching)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked language modeling (we want text generation)
)

Map:   0%|          | 0/3979 [00:00<?, ? examples/s]

In [ ]:
### 4️. Fine-Tune GPT-2 ###
from sklearn.model_selection import train_test_split # Used to split the data into training data and evaluation data
import numpy as np

# Shuffle the indices so the split is not biased on alphabetical order
shuffled_indices = np.random.permutation(len(tokenized_datasets))

# Split the shuffled indices into training and evaluation indices
train_indices, eval_indices = train_test_split(
    shuffled_indices,
    test_size=0.2, # Use 20% of the data for evaluation
    random_state=42 # Seed for being able to reproduce the code and having fair comparisons when tuning hyperparameters
)

# Select the data based on the split indices
train_dataset = tokenized_datasets.select(train_indices)
eval_dataset = tokenized_datasets.select(eval_indices)

In [ ]:
training_args = TrainingArguments(
    output_dir="./distilgpt2-negative-words",  # Directory to save the model
    evaluation_strategy="epoch",
    learning_rate=5e-5, # Learning rate hyperparameter
    per_device_train_batch_size=8, # Batch size for training
    num_train_epochs=5,  # Adjust based on performance
    weight_decay=0.01, # Regularization hyperparameter
    save_steps=500, # Save the model every 500 steps
    logging_dir="./logs",
    report_to="none"
)

trainer = Trainer( # Trainer to train the model 
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()  # Start training

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-60-1537f6f3eef1>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,4.558622
2,3.399100,4.279694
3,2.333900,4.244341
4,2.016700,4.331030
5,2.016700,4.372269


TrainOutput(global_step=1990, training_loss=2.3953897907506283, metrics={'train_runtime': 4193.972, 'train_samples_per_second': 3.795, 'train_steps_per_second': 0.474, 'total_flos': 40610720563200.0, 'train_loss': 2.3953897907506283, 'epoch': 5.0})

In [ ]:
### 5️. Generate New Negative Words ###
def generate_negative_words(model, tokenizer, prompt=""):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors="pt")  # Convert text to input tensor

    output = model.generate(
        input_ids,
        max_length=4,  # Max length of the words. Specially useful as we are not passing the padding to the generating process.
        num_return_sequences=5,  # Return the top (5) words generated
        temperature=1.5,  # Higher values = more randomness
        #top_k=10,  # Limit choices to the top 10 words [Replaced with top_p]
        #num_beams=5 # works with beam search. Explore multiple (5) possible sequences. [Replaced with sampling]
        do_sample=True,  # Use sampling to choose words
        top_p=0.9 # Adjust the threshold
    )

    return [tokenizer.decode(generated_text, skip_special_tokens=True) for generated_text in output]

# Example: Generate negative words
new_words = generate_negative_words(model, tokenizer, " " ) # In " " enter the starting characters for the generated words
print("Generated Negative Words:", new_words)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Negative Words: [' iasicalisms', ' iliabilities of', ' usionalnesses in', ' ickynessese', ' islikesets']
